<a href="https://colab.research.google.com/github/flych3r/IA025_2022S1/blob/main/ex08/matheus_xavier/IA025_A08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = 'Matheus Xavier Sampaio - 220092'
print(f'Meu nome é {nome}')

Meu nome é Matheus Xavier Sampaio - 220092


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
%pip install -qqq transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -qqq wandb

Note: you may need to restart the kernel to use updated packages.


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [ ]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 20:30:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List
from transformers import AutoTokenizer


def tokenize(text: str, tokenizer: AutoTokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer: AutoTokenizer, context_size: int):
        self.examples = []
        for text in tqdm(texts):
            token_ids = tokenize(text=text, tokenizer=tokenizer)
            if len(token_ids) < context_size + 1:
                continue
            # Compute n-grams:
            for i in range(len(token_ids) - context_size):
                input_ids = token_ids[i:i + context_size]
                target_id = token_ids[i + context_size]
                self.examples.append((input_ids, target_id)) 

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids, target_id = self.examples[idx]
        return torch.LongTensor(input_ids), target_id

### Teste se sua implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor([
    [ 3396, 10303,   125],
    [ 1660,  5971,   785],
    [ 5971,   785,   125],
    [  785,   125,  1847],
    [  125,  1847, 13779]
])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')

assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


### Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-25 20:30:47--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.107.128, 173.194.215.128, 173.194.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   157MB/s    in 0.8s    

2022-05-25 20:30:49 (157 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [ ]:
# Load datasets
context_size = 9

texts = open('sample_brwac.txt').readlines()

# print('Truncating for debugging purposes.')
# texts = texts[:500]
valid_examples = 100
test_examples = 100

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

  0%|          | 0/24800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


## Modelo de Linguagem

In [ ]:
import math

class LanguageModel(torch.nn.Module):
    def __init__(
        self,
        vocab_size: int,
        context_size: int,
        embedding_dim: int,
        hidden_size: int = None
    ):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
            hidden_size (int): Size of the hidden layer.
        """
        # Escreva seu código aqui.
        super(LanguageModel, self).__init__()
        self.vocab_size = vocab_size
        self.context_size = context_size
        self.embedding_dim = embedding_dim
        if hidden_size is None:
            hidden_size = embedding_dim
        self._positions = torch.arange(self.context_size, dtype=torch.long).unsqueeze(0)
        
        self.C = torch.nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embedding_dim)
        self.P = torch.nn.Embedding(num_embeddings=self.context_size, embedding_dim=self.embedding_dim)

        self.Wq = torch.nn.Linear(in_features=self.embedding_dim, out_features=self.embedding_dim, bias=False)
        self.Wk = torch.nn.Linear(in_features=self.embedding_dim, out_features=self.embedding_dim, bias=False)
        self.Wv = torch.nn.Linear(in_features=self.embedding_dim, out_features=self.embedding_dim, bias=False)
        self.Wo = torch.nn.Linear(in_features=self.embedding_dim, out_features=self.embedding_dim, bias=False)

        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(in_features=self.embedding_dim, out_features=hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=hidden_size, out_features=self.vocab_size, bias=False)
        )
        
        self.softmax = torch.nn.Softmax(dim=-1)


    def forward(self, inputs: torch.Tensor):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
        
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        # Escreva seu código aqui.
        positions = self._positions.repeat(inputs.shape[0], 1).to(inputs.device)

        X = self.C(inputs) + self.P(positions)
        Q = self.Wq(torch.unsqueeze(X[:, -1, :], dim=1))
        K = self.Wk(X)
        V = self.Wv(X)

        scores = Q @ torch.transpose(K, 1, 2) / math.sqrt(self.embedding_dim)
        probs = self.softmax(scores)

        E = probs @ V
        E = self.Wo(E)
        logits = self.mlp(torch.squeeze(E, dim=1))
        return logits

### Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64
).to(device)

model.eval()
sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train = torch.stack([sample_train, sample_train]).squeeze(1)
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([2, 29794])

In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 3834752


## Laço de Treinamento e Validação

In [ ]:
import wandb
from copy import deepcopy

In [ ]:
wandb.init(project="language-models", anonymous="allow")
wandb.run.name = f'selfattention-{wandb.run.name}'

wandb: Currently logged in as: flych3r. Use `wandb login --relogin` to force relogin


In [ ]:
max_examples = 500_000_000
eval_every_steps = 1_000
lr = 3e-4
batch_size = 4096 * 3

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
    hidden_size=256
).to(device)

train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=batch_size)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0

best_ppl = torch.inf
best_model = deepcopy(model.state_dict())

wandb.watch(model, log_freq=100)
pbar = tqdm(total=max_examples)

while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)

        if step % eval_every_steps == 0:
            train_loss = np.average(train_losses)
            train_ppl = np.exp(train_loss)
    
            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader
                ]))
            
            wandb.log({
                "train/loss": loss,
                "train/perplexity": train_ppl,
                "eval/perplexity": valid_ppl
            }, step=step)
            if valid_ppl < best_ppl:
                best_ppl = valid_ppl
                best_model = deepcopy(model.state_dict())
                torch.save(best_model, 'best_model.pth')
                
                artifact = wandb.Artifact(
                    'model-selfattention',
                    type='model',
                    metadata={
                        "step": step,
                        "step_size": batch_size,
                        "train_loss": train_loss,
                        "train_perplexity": train_ppl,
                        "valid_perplexity": valid_ppl
                    }
                )
                artifact.add_file('best_model.pth')
                wandb.run.log_artifact(artifact)

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        pbar.update(len(input))
        if n_examples >= max_examples:
            break

  0%|          | 0/500000000 [00:00<?, ?it/s]

0 steps; 0 examples so far; train ppl: 29827.77, valid ppl: 29637.71
1000 steps; 12288000 examples so far; train ppl: 1912.20, valid ppl: 1794.37
2000 steps; 24576000 examples so far; train ppl: 1430.30, valid ppl: 1105.06
3000 steps; 36864000 examples so far; train ppl: 938.95, valid ppl: 851.88
4000 steps; 49152000 examples so far; train ppl: 726.68, valid ppl: 622.31
5000 steps; 61440000 examples so far; train ppl: 548.98, valid ppl: 499.93
6000 steps; 73728000 examples so far; train ppl: 460.66, valid ppl: 434.98
7000 steps; 86016000 examples so far; train ppl: 409.07, valid ppl: 393.38
8000 steps; 98304000 examples so far; train ppl: 373.51, valid ppl: 364.92
9000 steps; 110592000 examples so far; train ppl: 350.47, valid ppl: 342.77
10000 steps; 122880000 examples so far; train ppl: 327.77, valid ppl: 326.87
11000 steps; 135168000 examples so far; train ppl: 314.50, valid ppl: 312.52
12000 steps; 147456000 examples so far; train ppl: 299.53, valid ppl: 302.58
13000 steps; 1597440

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 188.9667604164408


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompts = [
    'Eu gosto de comer pizza pois me faz',
    'Cachorro e gato são os melhores amigos do',
    'Eu gosto de comer chocolate, pois me faz',
    'Campinas é uma cidade grande e eu gosto de',
    'As plantações de cana no Brasil desse ano serão',
    'As mulheres estão cada vez mais presentes comparado a',
    'Temos que pensar no futuro e guardar o que aprendemos na',
    'Uma xícara de café e um livro são',
    'Viagens de avião são muito mais rápidas, porém',
]
max_output_tokens = 20

model.eval()
for prompt in prompts:
    for _ in range(max_output_tokens):
        input_ids = tokenize(text=prompt, tokenizer=tokenizer)
        input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
        logits = model(torch.LongTensor([input_ids_truncated]).to(device))
        # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
        # Isso se chama decodificação gulosa (greedy decoding).
        predicted_id = torch.argmax(logits).item()
        input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
        prompt = tokenizer.decode(input_ids)
        print(prompt)
    print()

Eu gosto de comer pizza pois me faz com
Eu gosto de comer pizza pois me faz com o
Eu gosto de comer pizza pois me faz com o que
Eu gosto de comer pizza pois me faz com o que eu
Eu gosto de comer pizza pois me faz com o que eu não
Eu gosto de comer pizza pois me faz com o que eu não tenho
Eu gosto de comer pizza pois me faz com o que eu não tenho certeza
Eu gosto de comer pizza pois me faz com o que eu não tenho certeza que
Eu gosto de comer pizza pois me faz com o que eu não tenho certeza que o
Eu gosto de comer pizza pois me faz com o que eu não tenho certeza que o que
Eu gosto de comer pizza pois me faz com o que eu não tenho certeza que o que se
Eu gosto de comer pizza pois me faz com o que eu não tenho certeza que o que se refere
Eu gosto de comer pizza pois me faz com o que eu não tenho certeza que o que se refere ao
Eu gosto de comer pizza pois me faz com o que eu não tenho certeza que o que se refere ao lado
Eu gosto de comer pizza pois me faz com o que eu não tenho certeza que 

In [ ]:
wandb.finish()

eval/perplexity,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/perplexity,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/perplexity,200.16263
train/loss,5.17382
train/perplexity,181.3078
